### Version History
+ Ver 1.0 : vgg net 구현
+ Ver 1.1 : encapsulation 및 refactoring 진행함

In [1]:
%matplotlib inline
!pip install tensorboardcolab

import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import tensorboardcolab
from tensorflow.python.training import checkpoint_utils as cp

Using TensorFlow backend.


## Load Image Data set

In [0]:
def load_cifar10() :
    # load cifar10 dataset 
    from keras.datasets import cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    # reshape (None, 1) -> (None)
    y_train, y_test = [np.reshape(y_train, [-1]), np.reshape(y_test, [-1])]

    # normalization 
    x_train, x_test = [(x_train-x_train.max()) / (x_train.max()-x_train.min()),
                         (x_test-x_test.max()) / (x_test.max()-x_test.min())]

    temp = x_train
    ratio = int(len(x_train) * 0.7)
    ratio_end = int(len(x_train) * 1.0)  
    
    train_image = temp[0:ratio, :, :, :]
    valid_image = temp[ratio:ratio_end , :, :, :]
    
    train_label = y_train[0:ratio]
    valid_label = y_train[ratio:ratio_end ]
    
    return train_image, train_label, valid_image, valid_label, x_test, y_test

## Data Augmentation

In [0]:
def random_crop_and_pad(images, pad=4) :
    '''
    pad 크기만큼 무작위로 위아래, 좌우로 움직이는 메소드
    '''
    
    _, h, w, _ = images.shape
    pad_images = np.pad(images, [(0,0), (pad, pad), (pad, pad), (0,0)],
                         mode = 'constant')
    crops = []
    for idx, image in enumerate(pad_images) :
        start_y = np.random.randint(0, pad)
        start_x = np.random.radinit(0, pad)
        cropped = image[start_y:start_y+h, start_x:start_x+w]
        crops.append(cropped)
    return np.stack(crops)

def random_flip_left_right(images) :
    '''
    무작위로 이미지를 좌우로 뒤집어 주는 메소드
    '''
    
    for idx, image in enumerate(images) :
        if np.random.random() > 0.5 :
            images[idx] = images[:,::-1]
    return images

def random_rotate(image, max_angle=15) :
    '''
    최대 max angle내에서 이미지를 무작위로 회전시키는 메소드
    '''
    
    _, h, w, _ = images.shape
    
    for idx, images in enumerate(images) :
        angle = np.random.randint(-max_angle, max_angle)
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
        images[idx] = cv2.warpAffine(image, M, w, h)
    return images

def random_brightness_correction(image, min_val=0.8, max_val=1.2) :
    '''
    무작위로 Brightness값을 바꿔주는 메소드
    '''
    
    results = []
    for image in images :
        value = np.random.uniform(min_val, max_val)
        image = cvv2.cvtCOlor(image, cv2.COLOR_RGB2RSV)
        results.append(image)
    return np.stack(result)

def augment_images(images) :
    '''
    평행이동 -> 뒤집기 -> 회전 -> 명도 변환을 거쳐 데이터에 노이즈를 주는 메소드
    '''
    
    images = random_crop_and_pad(images, pad=4)
    images = random_flip_left_right(images)
    images = random_rotate(images)
    images = random_brightness_correction(images)
    return images
    

## Data Provider

In [0]:
class DataProvider(object):
    def __init__(self, x, y):
        self.epoch_count = 0
        
        self.data = x
        self.label = y
        
        npr.seed(42)
        
        self.indices = self.generate_indices()
        
    def generate_indices(self):
        indices = list(range(len(self.data)))
        npr.shuffle(indices)
        
        return indices
    
    def next_batch(self, batch_size):
        idx = batch_size
        if len(self.indices) < batch_size:
            print("all data consumed, epoch + 1")
            self.epoch_count += 1
            self.indices = self.generate_indices()
    
        target_indices = self.indices[:batch_size]
        del self.indices[:batch_size]
        
        return self.data[target_indices] , self.label[target_indices]

In [0]:
def cifar_generator(data, labels, batch_size=32):
    start_idx = 0
    num_step = len(data) // batch_size
    indexes = np.arange(0, len(data))
    while True:
        if start_idx >= num_step-1:
            np.random.shuffle(indexes)
            start_idx = 0
        else:
            start_idx += 1            
        batch_index = indexes[start_idx*batch_size: (start_idx+1)*batch_size]

        batch_data = data[batch_index]
        batch_label = labels[batch_index]

        yield batch_data, batch_label

## Load pretrained variables

In [0]:
def get_trained_weights(graph, sess) :
    
    # loading pretrained files
    from google.colab import drive
    drive.mount('/content/gdrive')
    
    !mkdir ./model
    !cp gdrive/My\ Drive/vgg/* model/ # from, to 임
    
    with graph.as_default() : 
        lode_dir = "./model/vgg_net_model_a"
        saver = tf.train.import_meta_graph(lode_dir + '.meta')
        saver.restore(sess, save_path = lode_dir)    
        
        reuse_vars = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        
        # trained variable name : values
        reuse_vars_dict = dict([(var.name.replace(':0',''), sess.run(var.name)) 
                                for var in reuse_vars])
        return reuse_vars_dict       

## model functions

In [0]:
def conv2d_init(input, in_, out_, floor, is_train, model, name) :
    for i in range(floor) :             
        kernel_init = tf.random.normal(shape=[2, 2, in_, out_], 
                                       mean=0.0, stddev=0.1, dtype=tf.float32) 
        bias_init   = tf.zeros([out_])
        kernel      = tf.Variable(kernel_init, name = name+'_kernel'+str(i+1))               
        bias        = tf.Variable(bias_init, name = name+'_bias'+str(i+1))        
        layer       = tf.nn.conv2d(input, kernel, strides=[1,1,1,1], 
                                   padding='SAME')
        layer       = layer + bias
        layer       = tf.nn.relu(layer)
        
        if model == "BN" :
            layer = tf.layers.BatchNormalization()(layer, training=is_train)
        
    return layer

In [0]:
def max_pooling2d(input) :
    pool  = tf.layers.MaxPooling2D(pool_size=[2,2], strides=[2,2])(input) 
    
    return pool

In [0]:
def fc_init(input, in_, out_, is_train, name) :
    he_init     = tf.sqrt(2/out_) # tf.initializers.he_normal()
    kernel_init = tf.random.normal(shape = [in_, out_], 
                                   mean=0.0, stddev=he_init, dtype = tf.float32)
    bias_init   = tf.zeros([out_])    
    kernel      = tf.Variable(kernel_init, name = name+"_kernel1")
    bias        = tf.Variable(bias_init, name = name+"_bias1")
    z           = tf.matmul(input, kernel) + bias 
    logits      = tf.nn.relu(z)
    
    if model == "BN" :
        logits = tf.layers.BatchNormalization()(logits, training=is_train)
    dropout     = tf.layers.Dropout(0.5)(logits, training = is_train)
    
    return dropout

In [0]:
def softmax_l2_with_loss(ys_true, ys_pred, weight_decay) :  
    sce_loss = tf.reduce_mean(
        tf.losses.sparse_softmax_cross_entropy(labels=ys_true, logits=ys_pred))
    l2_loss  = tf.add_n([tf.nn.l2_loss(var) for var in tf.global_variables()])
    loss     = sce_loss + weight_decay * l2_loss
    
    return loss

In [0]:
def accuracy(y_true, y_pred) :
    pred     = tf.cast(tf.arg_max(y_pred, 1), tf.int32)
    correct  = tf.equal(pred, y_true)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    # add tensor to tensorboard
    acc_tb   = tf.summary.scalar(name='accuracy', tensor=accuracy)
    
    return accuracy

## main model

In [0]:
graph = tf.Graph()

In [13]:
#class Conv_net() :
#    def __init__(self, graph, sess) :
#        # super().__init__()
#        self.graph = graph
#        self.sess = sess
#        self.sess.run(tf.global_variables_initializer())
#        print("initial is complete!")
#        
#    def train(self, image, label, lr) :
# def train() :
with graph.as_default() :
    xs       = tf.placeholder(tf.float32, (None, 32, 32, 3), name='xs') 
    ys       = tf.placeholder(tf.int32, (None), name='ys')
    lr       = tf.placeholder_with_default(0.001, (), name='lr')
    wd       = tf.placeholder_with_default(0.9, (), name='wd')
    is_train = tf.placeholder_with_default(False, (), name='is_train')
    m        = tf.placeholder_with_default(0.9, (), name='momentum')
    dr       = tf.placeholder_with_default(0.9, (), name='dropout_ratio')   
    model    = tf.placeholder_with_default("VGG", (), name='model')         
    # trained_dict = get_trained_weights(self.graph, self.sess)
    
    with tf.name_scope('VGGBlock-1') :
        layer = conv2d_init(xs, 3, 32, 1, is_train, model, 'conv1')
        # layer = conv2d(xs, trained_dict, is_train, 1, 'conv1')
        pool  = max_pooling2d(layer)   
        
    with tf.name_scope('VGGBlock-2') :
        layer = conv2d_init(pool, 32, 64, 1, is_train, model, 'conv2')
        pool  = max_pooling2d(layer)
        
    with tf.name_scope('VGGBlock-3') :
        layer = conv2d_init(pool, 64, 128, 1, is_train, model,'conv3')
        layer = conv2d_init(layer, 128, 128, 2, is_train, model,'conv3')
        pool = max_pooling2d(layer)
        
    with tf.name_scope('VGGBlock-4') :
        layer = conv2d_init(pool, 128, 256, 1, is_train, model,'conv4')
        layer = conv2d_init(layer, 256, 256, 2, is_train, model,'conv4')
        pool  = max_pooling2d(layer)
    
    with tf.name_scope('fc') : 
        flatten = tf.layers.flatten(pool)
        layer = fc_init(flatten, 1024, 1024, is_train, 'fc1')    
        layer = fc_init(layer, 1024, 1024, is_train, 'fc2')                                 
        layer = fc_init(layer, 1024, 512, is_train, 'fc3')
    
    with tf.name_scope('output') : 
        y_pred  = tf.layers.Dense(10, 
                                  activation=None, 
                                  name='y_pred')(layer)     
    
    with tf.name_scope('Loss') :
        loss = softmax_l2_with_loss(ys, y_pred, wd)
    loss = tf.identity(loss, name='loss')
        
    with tf.name_scope('metric') :
        rmse = tf.sqrt(loss)
    rmse = tf.identity(rmse, name='rmse')
    
    with tf.name_scope('accuracy') :
        acc = accuracy(ys, y_pred)
    acc = tf.identity(acc, name='acc')

    with tf.name_scope('train') :
        optimizer   = tf.train.AdamOptimizer(lr)
        train_op    = optimizer.minimize(loss)

W0621 02:56:44.101129 140360801970048 deprecation.py:323] From <ipython-input-13-d00926e59b2e>:32: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0621 02:56:44.468838 140360801970048 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0621 02:56:44.880856 140360801970048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has th

## pretrainingmodel

In [0]:
def model_a(config) :
    
    # load data    
    train_image, train_label, \
    valid_image, valid_label, \
    test_image, test_label = load_cifar10()
    
    # load hyper parameters
    model_type    = config['model_type']
    load_model    = config['load_model']
    save_model    = config['save_model']
    learning_rate = config['learning_rate']  
    batch_size    = config['batch_size']
    n_epoch       = config['epoch']
    n_step        = int(len(train_image) // batch_size)
    weight_decay  = config['weight_decay']
    dropout_ratio = config['dropout_ratio']
    
    # save directory 
    if load_model == None :
         lode_dir = None
    else :
         lode_dir = "./model/vgg_net_model_" + load_model  
    save_dir = "./model/vgg_net_model_" + save_model  
    log_dir = "./log/vgg_net_model_" + save_model
    
    # tensorboard
    tbc = tensorboardcolab.TensorBoardColab(graph_path = log_dir)
    train_writer = tf.summary.FileWriter(logdir = log_dir)
    train_writer.add_graph(tf.get_default_graph())
    merged_all = tf.summary.merge_all()    
    
    # graph = tf.Graph()
    with graph.as_default() :
        saver           = tf.train.Saver()    
        sess            = tf.Session()
        
        # create Instance
        # model_b         = Conv_net(graph, sess)
        train_generator = cifar_generator(train_image, train_label, batch_size)
        sess.run(tf.global_variables_initializer())
        
        # trained_dict = get_trained_weights(graph, sess)
    
        train_loss = []
        train_acc = []
        valid_loss = []
        valid_acc = []
        cnt = 0
        maximum_acc = 0.5
        for i in tqdm(range(n_epoch)) :
            for step in range(n_step) :
                batch_xs, batch_ys = next(train_generator)
                _, train_loss_, train_acc_ = sess.run([train_op, rmse, acc], 
                                            feed_dict = { xs: batch_xs, 
                                                          ys: batch_ys, 
                                                          lr: learning_rate,
                                                          wd : weight_decay,
                                                          dr : dropout_ratio, 
                                                          model : model_type,  
                                                          is_train : True})
                train_loss.append(train_loss_)
                train_acc.append(train_acc_)
            
                # check validation set
                if step % 100 == 0 :
                    loss_, acc_ = sess.run([rmse, acc], 
                                          feed_dict = { xs: valid_image, 
                                                        ys: valid_label,
                                                        wd : weight_decay,
                                                        is_train : False})
                    valid_loss.append(loss_)
                    valid_acc.append(acc_)
                
                    # Save the model
                    if acc_ > maximum_acc :
                        print("log current model! valid loss = {:.4f}, \
                               valid acc = {:.2f}%".format(loss_, acc_*100))
                        maximum_acc = acc_
                        saver.save(sess, save_path = save_dir)
            print(" valid loss = {:.4f}, valid acc = {:.2f}%". \
                  format(loss_, acc_*100))
        
        train_writer.flush() # file을 disk에 쓴다

## main function

In [15]:
def main() :
    
    # config of hyper parameters
    config = {
        "model_type"      : "VGG",  # VGG or BN
        "load_model"      : None,
        "save_model"      : "a",
        "learning_rate"   : 0.001,
        "batch_size"      : 1000,
        "epoch"           : 100,
        "weight_decay"    : 0.0005,
        "dropout_ratio"   : 0.5
    }
    
    # call reconstruct model
    model_a(config)
    
    
    # # config of hyper parameters
    # config = {
    #     "model_type"      : "BN",  # VGG or BN
    #     "load_model"      : None,
    #     "save_model"      : "a",
    #     "learning_rate"   : 0.001,
    #     "batch_size"      : 1000,
    #     "epoch"           : 100,
    #     "weight_decay"    : 0.0005,
    #     # Cifa10 Dataset은 overfitting이 심하기 때문에 dropout을 제거하는 대신 비율을 줄임
    #     "dropout_ratio"   : 0.4 
    # }
    
    # call reconstruct model
    # model_a(config)
    
if __name__ == '__main__':
    main()           

170500096/170498071 [==============================] - 2s 0us/step
Wait for 8 seconds...
TensorBoard link:
https://11d617dc.ngrok.io


  1%|          | 1/100 [00:09<15:35,  9.45s/it]

 valid loss = 2.5377, valid acc = 10.12%


  2%|▏         | 2/100 [00:12<12:20,  7.56s/it]

 valid loss = 2.2102, valid acc = 20.55%


...

 valid loss = 1.3973, valid acc = 68.66%


 99%|█████████▉| 99/100 [05:37<00:03,  3.37s/it]

 valid loss = 1.3951, valid acc = 68.52%
log current model! valid loss = 1.3823,                                valid acc = 69.10%


100%|██████████| 100/100 [05:40<00:00,  3.40s/it]

 valid loss = 1.3823, valid acc = 69.10%


## save model

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

!mkdir gdrive/My\ Drive/vgg
!mv ./model/vgg* gdrive/My\ Drive/vgg
!mv ./model/checkpoint gdrive/My\ Drive/vgg

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
mkdir: cannot create directory ‘gdrive/My Drive/vgg’: File exists
